In [1]:
!pip install datasets
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.4 MB/s eta 0:00:00


In [17]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
import torch
import random
import tensorflow as tf
from datasets import Dataset
import tqdm as notebook_tqdm
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

df_test = pd.read_csv("politicES_phase_2_test_codalab.csv", on_bad_lines='skip')
df_test.drop(['label','ideology_binary','gender','profession'], axis=1, inplace=True)
#indices_aleatorios = random.sample(range(len(df_test)), 30000)
#df_test = df_test.iloc[indices_aleatorios]


In [18]:
df_test

,ideology_multiclass,tweet
0,moderate_right,"Ayer, en Guadalajara: feministas arrojaban áci..."
1,moderate_right,"“Chile Vamos”, o sea, la centro-derechita coba..."
2,moderate_right,"Se está tratando de silenciar que en Bolivia, ..."
3,moderate_right,"@user Lucía, lo siento pero no es así. Hebe ja..."
4,moderate_right,JxC no dio quórum para eliminar el impuesto a ...
...,...,...
43755,right,"No, si ahora resulta que todo el mundo es prim..."
43756,right,Tenemos un buen futuro si entre todos nos pone...
43757,right,Cuando el [POLITICAL_PARTY] dice la estupidez ...
43758,right,La entrevista de @user a @user hoy en El Mundo...


In [19]:

# Initialize the tokenizer
checkpoint_name= 'juan-glez29/ideology-BERTuit-base'
tokenizer = AutoTokenizer.from_pretrained(checkpoint_name)
# Initialize the model
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_name)

In [20]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df_test['ideology_multiclass'] = label_encoder.fit_transform(df_test['ideology_multiclass'])

In [21]:
df_test['tweet'] = df_test['tweet'].apply(lambda x: x.replace("@user", "<usr>"))

In [22]:
def tokenize_function(example):
    return tokenizer(example['tweet'])

# Conversión del DataFrame de pandas a un Dataset de Hugging Face para pasarlo al modelo
huggingface_dataset = Dataset.from_pandas(df_test)
huggingface_dataset

Dataset({
    features: ['ideology_multiclass', 'tweet'],
    num_rows: 43760
})

In [23]:
# Aplicamos la tokenización a todo el dataset
tokenized_dataset = huggingface_dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns("tweet")
test_dataset = tokenized_dataset.rename_column("ideology_multiclass", "labels")

Map:   0%|          | 0/43760 [00:00<?, ? examples/s]

In [24]:
test_dataset

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 43760
})

In [25]:
# Define test trainer
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
test_trainer = Trainer(model, data_collator=data_collator, tokenizer=tokenizer)

predicted_results = test_trainer.predict(test_dataset)

# Preprocess raw predictions
y_pred = predicted_results.predictions.argmax(-1) # Get the highest probability prediction

# Calculate accuracy
labels = test_dataset["labels"]
print(f"Accuracy:\t{accuracy_score(y_true=labels, y_pred=y_pred)}")

precision, recall, f1, _ = precision_recall_fscore_support(y_true=labels, y_pred=y_pred, average='weighted') # o weighted si estan desbalceadas las clases
print(f"Precision:\t{precision}")
print(f"Recall:\t\t{recall}")
print(f"F1-Score:\t{f1}")

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Accuracy:	0.45754113345521025
Precision:	0.45292415150015347
Recall:		0.45754113345521025
F1-Score:	0.44770810377529613


In [33]:
# Tokenize text inputs
tokenized_inputs = tokenizer(df_test['tweet'][6], return_tensors="pt")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
tokenized_inputs = tokenized_inputs.to(device)

# Pass tokenized inputs through the Transformer model
outputs = model(**tokenized_inputs)

# Get predictions
labels = ['izq', 'izq_mod', 'dcha_mod', 'dcha']
prediction = torch.argmax(outputs.logits)

print(outputs)

print(f"The prediction is {labels[prediction]}")

SequenceClassifierOutput(loss=None, logits=tensor([[-0.3460, -1.3533,  0.8450,  0.5875]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
The prediction is dcha_mod
